In [13]:
import pandas as pd

In [14]:
df = pd.read_csv("../data/records.csv")
df.shape, df.dropna().shape

((814226, 6), (13082, 6))

In [15]:
df.head()

,original_id,image_url,long_title,description,artist_uri,author_name
0,https://id.rijksmuseum.nl/2001,https://iiif.micr.io/KMFvF/full/max/0/default.jpg,"Before, Behind, Between, Above, Below",Links naast de vrouw staat een paal waaraan ee...,https://id.rijksmuseum.nl/21029638,Nicolaas Wijnberg
1,https://id.rijksmuseum.nl/20010,https://iiif.micr.io/wgIZp/full/max/0/default.jpg,Badende jongens,In het midden trekt een jongen zijn hemd over ...,https://id.rijksmuseum.nl/21029638,Nicolaas Wijnberg
2,https://id.rijksmuseum.nl/20010000,https://iiif.micr.io/owHmZ/full/max/0/default.jpg,"Portret van Dirk I, graaf van Holland","Portret van Dirk I, graaf van Holland, staande...",NaN,NaN
3,https://id.rijksmuseum.nl/200100000,https://iiif.micr.io/NVgFC/full/max/0/default.jpg,"Peso van 8 reales uit de republiek Mexico,1863...",Zilveren munt. Voorzijde: op takken staande ad...,NaN,NaN
4,https://id.rijksmuseum.nl/200100001,https://iiif.micr.io/PPxWZ/full/max/0/default.jpg,Wilhelmina Koningin der Nederlanden,"Goudkleurig messing penninkje, op de voorzijde...",https://id.rijksmuseum.nl/21075739,NaN


In [16]:
df.author_name.value_counts(normalize=True, dropna=False)

author_name
NaN                             0.941004
Ed van der Elsken               0.001454
Printshop                       0.000960
Jeanne Bieruma Oosting          0.000942
Donker, Charles                 0.000749
                                  ...   
Jacob Duck                      0.000001
Mathieu Dubus                   0.000001
Huib van Hove Bz                0.000001
Wilhelmus Johannes Steenhoff    0.000001
Andrew Fairbairn Affleck        0.000001
Name: proportion, Length: 4574, dtype: float64